<a href="https://colab.research.google.com/github/LUMII-AILab/NLP_Course/blob/main/notebooks/MSP/speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install transformers[torch] yt-dlp gradio jiwer openai-whisper diff-match-patch

Thu Dec 11 15:26:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |    6274MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import pipeline
from IPython.display import Audio, display, HTML
import jiwer
import whisper
from whisper.tokenizer import get_tokenizer
from diff_match_patch import diff_match_patch
import re
from itertools import zip_longest
import gradio as gr

In [ ]:
# Piemēra audio
!wget https://share.viroling.lv/samples/diktats2024.mp3
!yt-dlp -x --audio-format mp3 --download-sections "*00:00:00-00:01:30" -o "zinas1.%(ext)s" "https://www.youtube.com/watch?v=ngEAnlTlNWI"

In [ ]:
# Utilities
def normalize(t):
    t = t.lower()
    t = re.sub(r'\W', ' ', t)
    t = re.sub(r'\s+', ' ', t)
    t = t.strip()
    return t

def show_aligment(pred, ref):
    """Char-level diff-match-patch: edits to turn pred -> ref."""
    dmp = diff_match_patch()
    diffs = dmp.diff_main(pred, ref)
    dmp.diff_cleanupSemantic(diffs)

    css = """<style>
      .d del{background:#ffe6e6;color:#7a1c1c;text-decoration:none;}
      .d ins{background:#e6ffed;color:#14532d;text-decoration:none;}
    </style>"""
    esc = lambda s: s.replace("&","&amp;").replace("<","&lt;").replace(">","&gt;")

    out = []
    for op, data in diffs:
        s = esc(data)
        if op == 0:   out.append(s)
        if op == -1:  out.append(f"<del>{s}</del>")
        if op == 1:   out.append(f"<ins>{s}</ins>")

    display(HTML(css + "<div class='d'>" + "".join(out) + "</div>"))

def show_wer_alignment(w):
  import html as _html
  boxes = []
  for a in w.alignments[0]:
      hyp_slice = w.hypotheses[0][a.hyp_start_idx:a.hyp_end_idx]
      ref_slice = w.references[0][a.ref_start_idx:a.ref_end_idx]
      for hyp_tok, ref_tok in zip_longest(hyp_slice, ref_slice, fillvalue=''):
          hyp = _html.escape(hyp_tok or '∅')
          ref = _html.escape(ref_tok or '∅')
          t = a.type

          boxes.append(f"""
            <div class="align-box align-{t}">
              <div class="hyp">{hyp}</div>
              <div class="ref">{ref if t != 'equal' else '&nbsp;'}</div>
            </div>
          """)
  html = """
  <style>
  # .align-wrap { font-family: monospace; }
  .align-box {
    display: inline-block;
    border: 1px solid #eee;
  }
  .align-insert     { background: #89ff7d; }
  .align-delete     { background: #ff9898; }
  .align-substitute { background: #fffaf0; }
  </style>
  <div class="align-wrap">
  """ + "\n".join(boxes) + "</div>"

  display(HTML(html))

def evaluate(ref, pred, normalizer=normalize):
    if normalizer:
        ref = normalizer(ref)
        pred = normalizer(pred)
    w = jiwer.process_words(ref, pred)
    c = jiwer.process_characters(ref, pred)
    print('WER', w.wer, "hits", w.hits, "subs", w.substitutions, "dels", w.deletions, "ins", w.insertions, "n_ref", sum(len(x) for x in w.references), "n_hyp", sum(len(x) for x in w.hypotheses))
    print('CER', c.cer, "hits", c.hits, "subs", c.substitutions, "dels", c.deletions, "ins", c.insertions, "n_ref", sum(len(x) for x in c.references), "n_hyp", sum(len(x) for x in c.hypotheses))
    show_wer_alignment(w)
    show_aligment(ref, pred)

In [ ]:
# Priekšskatīt audio
display(Audio('zinas1.mp3'))

# Oriģinālais whisper

| Size   | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|--------|------------|--------------------|---------------------|---------------|----------------|
| tiny   | 39 M       | tiny.en            | tiny                | ~1 GB         | ~10x           |
| base   | 74 M       | base.en            | base                | ~1 GB         | ~7x            |
| small  | 244 M      | small.en           | small               | ~2 GB         | ~4x            |
| medium | 769 M      | medium.en          | medium              | ~5 GB         | ~2x            |
| large  | 1550 M     | N/A                | large               | ~10 GB        | 1x             |
| turbo  | 809 M      | N/A                | turbo               | ~6 GB         | ~8x            |


In [ ]:
# Whisper no komandrindas
!whisper -h

usage: whisper [-h] [--model MODEL] [--model_dir MODEL_DIR] [--device DEVICE]
               [--output_dir OUTPUT_DIR]
               [--output_format {txt,vtt,srt,tsv,json,all}]
               [--verbose VERBOSE] [--task {transcribe,translate}]
               [--language {af,am,ar,as,az,ba,be,bg,bn,bo,br,bs,ca,cs,cy,da,de,el,en,es,et,eu,fa,fi,fo,fr,gl,gu,ha,haw,he,hi,hr,ht,hu,hy,id,is,it,ja,jw,ka,kk,km,kn,ko,la,lb,ln,lo,lt,lv,mg,mi,mk,ml,mn,mr,ms,mt,my,ne,nl,nn,no,oc,pa,pl,ps,pt,ro,ru,sa,sd,si,sk,sl,sn,so,sq,sr,su,sv,sw,ta,te,tg,th,tk,tl,tr,tt,uk,ur,uz,vi,yi,yo,yue,zh,Afrikaans,Albanian,Amharic,Arabic,Armenian,Assamese,Azerbaijani,Bashkir,Basque,Belarusian,Bengali,Bosnian,Breton,Bulgarian,Burmese,Cantonese,Castilian,Catalan,Chinese,Croatian,Czech,Danish,Dutch,English,Estonian,Faroese,Finnish,Flemish,French,Galician,Georgian,German,Greek,Gujarati,Haitian,Haitian Creole,Hausa,Hawaiian,Hebrew,Hindi,Hungarian,Icelandic,Indonesian,Italian,Japanese,Javanese,Kannada,Kazakh,Khmer,Korean,Lao,L

In [ ]:
!whisper zinas1.mp3 --model turbo

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Latvian
[00:00.000 --> 00:01.560]  SmartLink nelidos
[00:01.560 --> 00:06.160]  Novembrī darbību pārtrauc Latvijā bāsētā aviokompānija SmartLinks.
[00:06.340 --> 00:10.660]  Vēl pērni to atzina par vērtīgāko pašmāju uzņēmumu – transporta nozerē.
[00:10.800 --> 00:15.380]  SmartLinks ierosināja tiesiskās aizsardzības procesu, lai norēķinātos ar kreditoriem,
[00:15.660 --> 00:17.740]  taču trešdiena tiesisko aizsardzība izbeidza.
[00:18.120 --> 00:21.680]  Uzņēmums nav stagatavojis pasākumu plānu darbības turpināšanai,
[00:21.880 --> 00:24.700]  tās tuvina līdzsabiedrību maksāt nespējai.
[00:24.700 --> 00:29.300]  SmartLinks palikuši parāda vairāk nekā 230 miljonu eiro apmērā.
[00:29.680 --> 00:33.500]  Apkalpas locekļa ārzemēs sūdas par darba algas un nesaņemšanu.
[00:33.780 --> 00:37.520]  SmartLinks bija daļa no pasaulē, zināmā Aviesa Lūšanas grupa.
[00:37.740 --> 00:41.260

In [ ]:
!whisper zinas1.mp3 --model turbo --initial_prompt "SmartLynx"

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Latvian
[00:00.000 --> 00:01.580]  SmartLink nelidos
[00:01.580 --> 00:06.220]  Novembrī darbību pārtrauc Latvijā bāsētā avio kompānija SmartLynx.
[00:06.380 --> 00:10.660]  Vēl pērni to atzina par vērtīgāko pašmāju uzņēmumu transporta nozerē.
[00:10.800 --> 00:15.380]  SmartLynx ierosināja tiesiskās aizsardzības procesu, lai norēķinātos ar kreditoriem,
[00:15.660 --> 00:17.740]  taču trešdiena tiesisko aizsardzība izbeidza.
[00:18.120 --> 00:21.680]  Uzņēmums nav stagatavojis pasākumu plānu darbības turpināšanai,
[00:21.880 --> 00:24.700]  tās tuvina līdzsabiedrību maksāt nespējai.
[00:24.700 --> 00:29.300]  SmartLynx palikuši parāda vairāk nekā 230 miljonu eiro apmērā.
[00:29.680 --> 00:33.500]  Apkalpas locekļa ārzemēs sūdas par darba algas un nesaņemšanu.
[00:33.800 --> 00:37.520]  SmartLynx bija daļa no pasaulē, zināmā Avieselūšanas grupa.
[00:37.740 --> 00:41.260]  Šis

In [ ]:
!whisper zinas1.mp3 --model tiny

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 187MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Latvian
[00:00.000 --> 00:02.000]  Smarflink, mēlīdu.
[00:02.000 --> 00:06.240]  No vēm rī darbīb pāti alts Latvijā bāsētā avi akompānīs Smarflink.
[00:06.240 --> 00:10.720]  Dao pērnto azim par vētīgā ko pašmāju uzņemam transportenu uzērē.
[00:10.720 --> 00:15.520]  Smarflink 090 iskāsēs ir zīb procesu laidnūrēti nātos ir kreditārēm.
[00:15.520 --> 00:17.920]  Peču drždēntiesas ko eser zībis bēd.
[00:17.920 --> 00:21.760]  Uzņemams nau sāgātu vēs pāsā komplāndarbīb subti nāšānē.
[00:21.760 --> 00:24.960]  Tas tuvina, lītza biedrībūt māksātnēs pēja.
[00:24.960 --> 00:29.520]  Smarflinks pāli kus pārādvērāk nekā 23.000 milionērāp mērā.
[00:29.520 --> 00:33.680]  Abkalbūs lūt sākljārzēmēs suds pardārbāloga sēnēs niemšāl.
[00:33.680 --> 00:37.600]  Smarflink 090 iskās pāsālē, zināmā āvijas ilusšens

In [ ]:
# The turbo model will return the original language even if --task translate is specified. Use medium or large for the best translation results.
!whisper zinas1.mp3 --model large-v3 --language lv --task translate

[00:00.000 --> 00:02.000]  SmartLink is not flying
[00:02.000 --> 00:06.000]  In November, the company SmartLinks was transferred to Latvia.
[00:06.000 --> 00:11.000]  The company has already announced that the most valuable home investment is in transport.
[00:11.000 --> 00:15.000]  SmartLinks has completed the legal protection process in order to deal with creditors.
[00:15.000 --> 00:18.000]  However, the third day of legal protection has ended.
[00:18.000 --> 00:22.000]  The company has not yet prepared for the start of planning operations.
[00:22.000 --> 00:25.000]  It is not possible to pay the rent to the municipality.
[00:25.000 --> 00:29.000]  SmartLinks has left more than EUR 230 million behind.
[00:29.000 --> 00:33.000]  Surroundings have been closed due to lack of employment.
[00:33.000 --> 00:37.000]  SmartLinks was part of the well-known Aviesolutions Group in the world.
[00:37.000 --> 00:41.000]  This company is making full use of the service aircraft NOMU.
[00:41.000 --

In [ ]:
# Original whisper
m = whisper.load_model("turbo")

In [ ]:
audio = whisper.load_audio("zinas1.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=m.dims.n_mels).to(m.device)

# detect the spoken language
_, probs = m.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(m, mel, options)

tokenizer = get_tokenizer(True)
print('Decoded', [tokenizer.encoding.decode([t]) for t in result.tokens])

# print the recognized text
print(result)
print(result.text)

Detected language: lv
Decoded ['<|0.02|>', ' Smart', 'Link', ' nel', 'idos', '<|1.58|>', '<|1.58|>', ' Nove', 'm', 'br', 'ī', ' dar', 'b', 'ī', 'bu', ' p', 'ā', 'r', 'tra', 'uc', ' Lat', 'v', 'ij', 'ā', ' b', 'ā', 's', 'ē', 't', 'ā', ' av', 'i', 'ok', 'omp', 'ān', 'ija', ' Smart', 'L', 'inks', '.', '<|6.18|>', '<|6.36|>', ' V', 'ē', 'l', ' p', 'ē', 'r', 'ni', ' to', ' at', 'z', 'ina', ' par', ' v', 'ē', 'rt', 'ī', 'g', 'ā', 'ko', ' pa', 'š', 'm', 'ā', 'ju', ' uz', 'ņ', 'ē', 'm', 'umu', ' transport', 'a', ' no', 'zer', 'ē', '.', '<|10.68|>', '<|10.82|>', ' Smart', 'L', 'inks', ' i', 'eros', 'in', 'ā', 'ja', ' ties', 'isk', 'ā', 's', ' a', 'iz', 's', 'ard', 'z', 'ī', 'bas', ' proces', 'u', ',', ' la', 'i', ' nor', 'ē', '�', '�', 'in', 'ā', 't', 'os', ' ar', ' k', 'red', 'itor', 'iem', ',', '<|15.40|>', '<|15.68|>', ' ta', 'č', 'u', ' tre', 'š', 'd', 'ien', 'a', ' ties', 'isko', ' a', 'iz', 's', 'ard', 'z', 'ī', 'ba', ' iz', 'be', 'id', 'za', '.', '<|17.76|>', '<|18.14|>', ' Uz', 'ņ', 'ē'

# Lietošana no HF

In [ ]:
display(Audio('diktats2024.mp3'))

In [ ]:
ref = """Aiva Kanepone
Putni brīnās par Liepāju
Kurš liepājnieks gan nezina, ka Liepājā, Zirgu salā, ir putnu vērošanas tornis? Bet kurš ir pamanījis, ka arī putni vēro Liepāju? Brīnās, bola acis un groza galvas, un bezkaunīgi planē pāri kafejnīcu galdiņiem, lai ieskatītos nelidojošo divkāju šķīvjos. Liepājā ir daudz noslēpumaina, par ko brīnīties. Piemēram, naktīs, kad ne tikvien mēness piesprausts pie debesu samta kā zelta ananass, bet arī zvaigznes izbārstītas no Dieva atvilktnēm. Tad tur, lejā, zīriņiem par brīnumu, kaut kas iekvēlojas kā milzīgs ugunskurs, un tam viducī vītero kanārijputniņu bars. „Kas tas ir? Kas tas ir?” sakliedzas kaijas. Tām neviens nav izstāstījis par koncertzāli „Lielais dzintars” un simfonisko orķestri. Vēl viņi nesaprot, kāpēc jaukie Liepājas baznīcu torņi ik pa laikam sajūk prātā un pēkšņi sākas tāda dārdoņa, it kā tajos būtu ielauzušies kazuāri: „Bom! Bom! Bom!”
Un kas notiek Graudu ielas smalkāko namu augstumos? Tur nometas estēti baloži, kas dievina jūgendstilu. Ak, tie lieliskie izciļņi, kur dūdojot griezt mīlas dejas, un iedobumi, kur sastiept liepās salauztos zariņus minimālisma stilā ieturētajām ligzdām!
Kormorāni, viltnieki, jau trešo gadu perina bruņotu laupīšanas uzbrukumu Zivju tirgum Liepājas promenādē, tikai nevar sagaidīt, kad to atkal atvērs. Čurkstes slepeni skatās Liepājas teātra izrādes, bet pēc tam sajūsmā savidžinās. Bet zivju gārņi sapņo, garās kājas cilājot, pastaigāt pa Rožu laukuma dobēm. Diemžēl gārņi kautrējas. Toties vārnai Liepājā ir piemineklis. Karr!
Visskaistākajos vakaros, spārnus plivinādami, putni plecu pie pleca salaižas uz Ziemeļu mola vērot saulrietu. Kormorāns blakus paugurknābja zosij, cekulpīle blakus purva ķauķim, un !pašā galā tārtiņš, no Līvu krasta atlaidies, klusiņām purpina lībiešu valodā.
Rudenī viņi lūr pa Liepājas skolu logiem. Ieraugot gramatiskās kļūdas, saķer apaļās galveles un spalgi čiepst priekšā: „Stulbs nav stūlbs, albums nav albūms, mobilais nav nekāds mobīlais! Čiv!”
Klau, vai tagad arī uz palodzes negrabinās viena zilzīlīte? Ko, ko viņa tur čiepst?
"""

### Novērtējam openai/whisper-tiny

In [ ]:
m = pipeline(task="automatic-speech-recognition", model="openai/whisper-tiny")

Device set to use cuda:0


In [ ]:
pred = m('diktats2024.mp3', return_timestamps=True)
pred['text']

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


' Ajvākā neponē, potnī brīnās par liapājā. Kūs liapānieks gan nezdīnākā liapājā, zergu salā ir potnumerušanas tuvarnis. Bet kūs ir pamaneis, ka arī potnīt veru liapājā. Brīnās, būl, atsis, un grūs, galvas, un beskā unīgi planējā parikā feinīt, un galdiniem, lai eskatīt uz neliduojošu, djūkāju, štīvijos. Liepājā ir daudz, noslēpā mēn parku, brīnīties. Piemēram naktis, kad ne tikvien menes piesprosts pie, dabas asam pkā zaldā ananas, bet arī zvajīgsnesis bārstītas no dīja vātvilk nēm. Tattur, lēja, zīrņiem per brīnu kautkas iekvēlu eskā milzīgs ugunskurs. Un tam vidu cīvītoru a kanāri putnīņu bārstas. Kas tas ir, kas tas ir saklīdzis kājez. Tam nevienst navistāstējis per konserdzāli lielēs dinters un symfonis ko Ortestri. Vēl viņi nesaprūt kapēc jau kēlības, baznīt sturni, ikpalēkam sa jūk prata, un peksņu sakas tāda darduņa. Tajos būtu, ila uz uzšies, ka zlāri, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, b

In [ ]:
evaluate(ref, pred['text'])

WER 1.0436241610738255 hits 49 subs 245 dels 4 ins 62 n_ref 298 n_hyp 356
CER 0.6471774193548387 hits 1335 subs 565 dels 84 ins 635 n_ref 1984 n_hyp 2535


### Novērtējam openai/whisper-large-v3-turbo

In [ ]:
m = pipeline(task="automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
pred = m('diktats2024.mp3', return_timestamps=True)
pred['text']

' Aiva Kanepone putni brīnās par liepāju. Kurš liepājnieks gan nezina, ka liepājā zirgu salā ir putnu vērošanas tornis, bet kurš ir pamanījis, ka arī putni vēro liepāju. Brīnās bola acis un groza galvas Un bezkaunīgi planē pāri kafeinīcu galdiņiem Lai ieskatītos nelidojošo divkāju šķīvjos Liepājā ir daudz noslēpumainu par ko brīnīties Piemēram naktīs, kad ne tik vien mēnesis Piesprausts pie debesu samta kā zelta ananas bet arī zvaigznes izbārstītas no dieva atvilknēm. Tad tur, lejā, zīriņiem par brīnumu, kaut kas iekvēlojas kā milzīgs ugunskurs un tam viducī vītero kanārī putniņu bars. Kas tas ir? Kas tas ir? sakliedzis kā ejas. Tām neviens nav izstāstījis par koncertzāli lielais dzintars un simfonisko orķestri Vēl viņi nesaprot, kāpēc jaukie liepājas baznītes tūrņi ik pa laikam sajūk prātā Un pēkšņi sākas tāda dārdoņa, it kā tajos būtu ielauzušies kazuāri BOM! BOM! BOM! Un kas notiek graudu ielas smalkāko namu augstumos? Tur nometas estēti baloži, kas dievina jūgenstil. Ak, tie lielis

In [ ]:
evaluate(ref, pred['text'])

WER 0.13087248322147652 hits 263 subs 33 dels 2 ins 4 n_ref 298 n_hyp 300
CER 0.025705645161290324 hits 1945 subs 28 dels 11 ins 12 n_ref 1984 n_hyp 1985


### Novērtējam facebook/mms-1b-all

In [ ]:
m = pipeline(task="automatic-speech-recognition", model="facebook/mms-1b-all")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
pred = m('diktats2024.mp3', generate_kwargs={"language": "lav", "task": "transcribe"}, chunk_length_s=30, stride_length_s=5)
pred['text']

'aiva kanepone putni brins par liepju kur liepajnieks gan nezina ka liepaja zirgu sala ir putnu vroanas tornis bet kurs ir pamanijis ka ari putni vero liepaju brins bola acis un groza galvas un bezkaunigi plane pri kafeinicu galdiniem lai ieskatitos nelidojoo diukaju stivjos liepja ir daudz noslepumaini par ko brinties piemram naktis kad ne tikvien menests piesprausts pie debesa samta ka zelta ananass bet ari zvaigznes izbarstitas no dieva atvilknem tad tur leja ziriniem par brinumu kaut kas iekvelojas ka milzigs ugunskurs un tam viducivitero kanri putninu bars kas tas ir kas tas ir sakliedzas kailas tam neviens nav izststijis par koncertzali lielais dzinters un simfonisko ortestri vel vini nesaprot kapec jaukie liepajas baznicas turni ikpalaikam sajuk prata un peknis sakas tada dardona itk tajos butu ielauzuies kazuari bom bo bo un kas notiek graudu ielast smalkko namu augstumos tur nometas estti baloi kas dievina jugenstil ak tie lieliskie izcilni kur dudojot griezt milas dejas un ie

In [ ]:
evaluate(ref, pred['text'])

WER 0.5335570469798657 hits 140 subs 153 dels 5 ins 1 n_ref 298 n_hyp 294
CER 0.10887096774193548 hits 1774 subs 157 dels 53 ins 6 n_ref 1984 n_hyp 1937


## ASR demo saskarne

In [ ]:
m = pipeline(task="automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

In [ ]:
# To record from the microphone, you need to open the public link. Tested in Chrome.
def transcribe(audio):
    pred = m(audio, return_timestamps=True)
    print(pred)
    return pred.get("text", str(result))

gr.Interface(
    title="ASR demo",
    fn=transcribe,
    inputs=gr.Audio(sources=["microphone", "upload"], type="filepath"),
    outputs=gr.Textbox(),
).launch(share=True, debug=True)